In [1]:
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer
import pandas as pd
import numpy as np

## Daten einlesen

In [2]:

# Laden des Datensatzes
data_cleaned = "../data/cleaned_train_alt.csv"
data = pd.read_csv(data_cleaned, delimiter=",", encoding="latin", header=0, thousands=",", decimal='.', low_memory=False)

# Zielvariable und Features definieren
X = data.drop('Sales', axis=1)
y = data['Sales']

# Definiere die numerischen und kategorischen Features
numerical_features = ['year', 'month', 'day', 'week_of_year', 'lag_1', 'lag_7']

# Bereits encodierte Features
already_encoded_features = ['Open', 'Promo', 'promo2']

# Noch nicht encodierte kategorische Features
categorical_features_to_encode = ['Store', 'DayOfWeek', 'StoreType', 'StateHoliday','Assortment']


## Split

In [20]:

# 1. Datensatz aufteilen in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
df = pd.DataFrame(data)

# Bestimme die Grenze für 80% Training und 20% Test
train_size = int(len(df) * 0.8)

# Aufteilen in Trainings- und Test-Datensätze
train = df.iloc[:train_size]
test = df.iloc[train_size:]

## Metrik

In [3]:
# Angepasste RMSPE-Funktion, die Tage mit 0 Sales ignoriert
def rmspe(y_true, y_pred):
    # Nur Fälle berücksichtigen, bei denen y_true nicht 0 ist
    mask = y_true != 0
    y_true_filtered = y_true[mask]
    y_pred_filtered = y_pred[mask]
    
    return np.sqrt(np.mean(((y_true_filtered - y_pred_filtered) / y_true_filtered) ** 2))

# RMSPE als Scorer definieren
rmspe_scorer = make_scorer(rmspe, greater_is_better=False)

## Pre-Processing

In [4]:
# Erstelle den Preprocessor für numerische und kategorische Features (ohne Datumsextraktion)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features + already_encoded_features),  # Skalierung für numerische und bereits encodierte Features
        ('enc', 'passthrough', already_encoded_features),  # Bereits encodierte Features durchschleusen (keine weitere Transformation)
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_to_encode)  # Nur noch nicht encodierte Features encodieren
    ])


## Random Forest mit Pre-Processing Daten

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import make_scorer

# RMSPE Funktion definieren
def rmspe(y_true, y_pred):
    mask = y_true != 0
    y_true_filtered = y_true[mask]
    y_pred_filtered = y_pred[mask]
    return np.sqrt(np.mean(((y_true_filtered - y_pred_filtered) / y_true_filtered) ** 2))

# Scorer für Cross-Validation
rmspe_scorer = make_scorer(rmspe, greater_is_better=False)

# Daten in Training (80%) und Test (20%) aufteilen
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# TimeSeriesSplit für Cross-Validation
tscv = TimeSeriesSplit(n_splits=5)

# Teste verschiedene Werte für n_estimators
n_estimators_range = [1,5,10,20]
train_rmspe_scores = []
test_rmspe_scores = []

for n in n_estimators_range:
    # Erstelle die Pipeline mit dem aktuellen Wert für n_estimators
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', RandomForestRegressor(n_estimators=n, random_state=42))
    ])
    
    # Cross-Validation auf dem Trainingsdatensatz durchführen
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=tscv, scoring=rmspe_scorer)
    mean_cv_score = -np.mean(cv_scores)  # Da RMSPE negativ definiert ist
    
    # Trainiere das Modell auf dem gesamten Trainingsdatensatz
    pipeline.fit(X_train, y_train)
    
    # Vorhersagen auf dem Testdatensatz
    y_test_pred = pipeline.predict(X_test)
    
    # Berechne RMSPE für den Testdatensatz
    test_rmspe = rmspe(y_test, y_test_pred)
    
    # Speichere die Scores
    train_rmspe_scores.append(mean_cv_score)
    test_rmspe_scores.append(test_rmspe)
    
    # Ausgabe der Ergebnisse
    print(f"n_estimators: {n} - Train RMSPE: {mean_cv_score:.4f}, Test RMSPE: {test_rmspe:.4f}")

# Plotten der Ergebnisse
plt.figure(figsize=(10, 6))
plt.plot(n_estimators_range, train_rmspe_scores, label='Train RMSPE (Cross-Validation)', marker='o')
plt.plot(n_estimators_range, test_rmspe_scores, label='Test RMSPE', marker='o')
plt.xlabel('n_estimators')
plt.ylabel('RMSPE')
plt.title('RMSPE vs. n_estimators (TimeSeriesSplit)')
plt.legend()
plt.grid(True)
plt.show()


n_estimators: 1 - Train RMSPE: 0.2424, Test RMSPE: 0.2644
n_estimators: 5 - Train RMSPE: 0.2187, Test RMSPE: 0.2469


KeyboardInterrupt: 

In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score, train_test_split

# Erstelle die Pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(n_estimators=40, random_state=42))
])

# Optional: Cross-Validation mit RMSPE als Scorer
cv_scores = cross_val_score(pipeline, X, y, cv=5, scoring=rmspe_scorer)

print(f"RMSPE CV-Scores: {cv_scores}")
print(f"Mean RMSPE: {-np.mean(cv_scores)}")  # Negative Werte, da greater_is_better=False

RMSPE CV-Scores: [-0.16908628 -0.15583158 -0.30281814 -0.16420855 -0.24732529]
Mean RMSPE: 0.20785396741046566


### Evaluation

In [17]:
# Trainiere das Modell
pipeline.fit(X_train, y_train)

# Vorhersagen auf dem Testdatensatz
y_pred = pipeline.predict(X_test)

# RMSPE auf dem Testdatensatz berechnen
rmspe_score = rmspe(y_test, y_pred)
print(f"RMSPE auf dem Testdatensatz: {rmspe_score}")

RMSPE auf dem Testdatensatz: 0.14426840086304352


In [23]:
estimators = [10, 50, 100, 200, 300, 400, 500]
for e in estimators:
    # Erstelle die Pipeline
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', RandomForestRegressor(n_estimators=e, random_state=42))
    ])

    # Optional: Cross-Validation mit RMSPE als Scorer
    cv_scores = cross_val_score(pipeline, X, y, cv=5, scoring=rmspe_scorer)
    print(f"n_estimators: {e}")
    print(f"RMSPE CV-Scores: {cv_scores}")
    print(f"Mean RMSPE: {-np.mean(cv_scores)}")  # Negative Werte, da greater_is_better=False
   
   # Trainiere das Modell
    pipeline.fit(X_train, y_train)

    # Vorhersagen auf dem Testdatensatz
    y_pred = pipeline.predict(X_test)

    # RMSPE auf dem Testdatensatz berechnen
    rmspe_score = rmspe(y_test, y_pred)
    print(f"RMSPE auf dem Testdatensatz: {rmspe_score}")

    # Ausgabe der Ergebnisse
    print("")


n_estimators: 10
RMSPE CV-Scores: [-0.17075823 -0.15749867 -0.29701022 -0.1658843  -0.24551046]
Mean RMSPE: 0.20733237824285541
RMSPE auf dem Testdatensatz: 0.14675355789333175



## Random Forest tageweise Vorhersage (aktuelle Version)

In [8]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import make_scorer
from sklearn.model_selection import TimeSeriesSplit

# RMSPE Funktion definieren
def rmspe(y_true, y_pred):
    mask = y_true != 0
    y_true_filtered = y_true[mask]
    y_pred_filtered = y_pred[mask]
    return np.sqrt(np.mean(((y_true_filtered - y_pred_filtered) / y_true_filtered) ** 2))

# RMSPE als Scorer definieren
rmspe_scorer = make_scorer(rmspe, greater_is_better=False)

# Laden des Datensatzes
data_cleaned = "../data/cleaned_train_alt.csv"
data = pd.read_csv(data_cleaned, delimiter=",", encoding="latin", header=0, thousands=",", decimal='.', low_memory=False)

# Zielvariable und Features definieren
X = data.drop('Sales', axis=1)
y = data['Sales']

# Definiere die numerischen und kategorischen Features
numerical_features = ['year', 'month', 'day', 'week_of_year', 'lag_1', 'lag_7']
already_encoded_features = ['Open', 'Promo', 'promo2']
categorical_features_to_encode = ['Store', 'DayOfWeek', 'StoreType', 'StateHoliday', 'Assortment']

# Preprocessing Pipeline für numerische und kategorische Features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features + already_encoded_features),
        ('enc', 'passthrough', already_encoded_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_to_encode)
    ])

# Modell für RandomForest erstellen
model = RandomForestRegressor(n_estimators=20, random_state=42)

# Pipeline erstellen
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

# Funktion zur Vorhersage mehrerer Tage mit Feature-Update
def predict_multiple_days_with_updates(X_last, num_days, feature_update_rules):
    predictions = []
    X_next = X_last.copy()
    
    for day in range(num_days):
        # Vorhersage des nächsten Tages
        y_next_pred = pipeline.predict([X_next])[0]
        predictions.append(y_next_pred)
        
        # Aktualisiere die Lag-Features basierend auf der Vorhersage
        X_next['lag_1'] = y_next_pred  # Verschiebe die vorhergesagte Sales als lag_1
        X_next['lag_7'] = X_last['lag_1']  # Aktualisiere lag_7 mit den vorherigen lag_1 Werten
        
        # Aktualisiere andere Features gemäß den Regeln
        X_next['day'] += 1  # Verschiebe den Tag
        if X_next['day'] > 31:  # Beispiel: Überlauf des Tages
            X_next['day'] = 1
            X_next['month'] += 1  # Verschiebe den Monat bei Tageswechsel
        if X_next['month'] > 12:  # Überlauf des Monats
            X_next['month'] = 1
            X_next['year'] += 1
        
        # Aktualisiere Wochenzahl
        X_next['week_of_year'] = (X_next['week_of_year'] % 52) + 1
        
        # Aktualisiere den Tag der Woche
        X_next['DayOfWeek'] = (X_next['DayOfWeek'] % 7) + 1
        
        # Update Promo, Open, StateHoliday etc. basierend auf Regeln
        # Hier könnte man festlegen, wann Promo oder Feiertage kommen
        
        # Nutze die tatsächlichen zukünftigen Daten, falls vorhanden
        if feature_update_rules.get('Promo'):
            X_next['Promo'] = feature_update_rules['Promo'](X_next)

        # Optionale Updates für andere Features (Promo, Feiertage etc.)
    
    return predictions

# TimeSeriesSplit für Cross-Validation
tscv = TimeSeriesSplit(n_splits=5)

# Überprüfung mit Cross-Validation
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

cv_rmspe_scores = []
for train_idx, test_idx in tscv.split(X_train):
    X_train_cv, X_test_cv = X_train.iloc[train_idx], X_train.iloc[test_idx]
    y_train_cv, y_test_cv = y_train.iloc[train_idx], y_train.iloc[test_idx]
    
    # Pipeline auf den Cross-Validation-Trainingsdaten fitten
    pipeline.fit(X_train_cv, y_train_cv)
    
    # Vorhersage für den Testdatensatz
    y_pred_cv = pipeline.predict(X_test_cv)
    
    # Berechne RMSPE für diesen Split
    rmspe_score = rmspe(y_test_cv, y_pred_cv)
    cv_rmspe_scores.append(rmspe_score)

# Ausgabe der Ergebnisse
mean_cv_rmspe = np.mean(cv_rmspe_scores)
print(f"Durchschnittliche RMSPE (Cross-Validation): {mean_cv_rmspe:.4f}")

# Nimm die letzten Daten als Startpunkt für die Vorhersage
X_last = X_test.iloc[-1]

# Beispiel: Definiere Regeln zur Aktualisierung von Promo
feature_update_rules = {
    'Promo': lambda X: 1 if (X['day'] % 7 == 0) else 0  # Beispielregel: Promo jeden 7. Tag
}

# Vorhersage für die nächsten 7 Tage
num_days = 7
future_predictions = predict_multiple_days_with_updates(X_last, num_days, feature_update_rules)

# Ergebnisse anzeigen
print(f"Vorhersagen für die nächsten {num_days} Tage:", future_predictions)

Root Mean Square Percentage Error (RMSPE): inf


Werte für Promo aus Testdatensatz verwendet

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import make_scorer
from sklearn.model_selection import TimeSeriesSplit

# RMSPE Funktion definieren
def rmspe(y_true, y_pred):
    mask = y_true != 0
    y_true_filtered = y_true[mask]
    y_pred_filtered = y_pred[mask]
    return np.sqrt(np.mean(((y_true_filtered - y_pred_filtered) / y_true_filtered) ** 2))

# RMSPE als Scorer definieren
rmspe_scorer = make_scorer(rmspe, greater_is_better=False)

# Laden des Datensatzes
data_cleaned = "../data/cleaned_train_alt.csv"
data = pd.read_csv(data_cleaned, delimiter=",", encoding="latin", header=0, thousands=",", decimal='.', low_memory=False)

# Zielvariable und Features definieren
X = data.drop('Sales', axis=1)
y = data['Sales']

# Definiere die numerischen und kategorischen Features
numerical_features = ['year', 'month', 'day', 'week_of_year', 'lag_1', 'lag_7']
already_encoded_features = ['Open', 'Promo', 'promo2']
categorical_features_to_encode = ['Store', 'DayOfWeek', 'StoreType', 'StateHoliday', 'Assortment']

# Preprocessing Pipeline für numerische und kategorische Features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features + already_encoded_features),
        ('enc', 'passthrough', already_encoded_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_to_encode)
    ])

# Modell für RandomForest erstellen
model = RandomForestRegressor(n_estimators=1, random_state=42)

# Pipeline erstellen
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

# Funktion zur Vorhersage mehrerer Tage mit Feature-Update
def predict_multiple_days_with_updates(X_last, test_data, num_days):
    predictions = []
    X_next = X_last.copy()

    # Wandle X_next in ein DataFrame um, falls es keine DataFrame ist
    X_next = pd.DataFrame([X_next])

    for day in range(num_days):
        # Vorhersage des nächsten Tages
        y_next_pred = pipeline.predict(X_next)[0]
        predictions.append(y_next_pred)
        
        # Aktualisiere die Lag-Features basierend auf der Vorhersage
        X_next.loc[:, 'lag_1'] = y_next_pred  # Verschiebe die vorhergesagte Sales als lag_1
        X_next.loc[:, 'lag_7'] = X_last['lag_1']  # Aktualisiere lag_7 mit den vorherigen lag_1 Werten
        
        # Aktualisiere andere Features
        X_next.loc[:, 'day'] += 1  # Verschiebe den Tag
        if X_next.loc[:, 'day'].values[0] > 31:  # Beispiel: Überlauf des Tages
            X_next.loc[:, 'day'] = 1
            X_next.loc[:, 'month'] += 1  # Verschiebe den Monat bei Tageswechsel
        if X_next.loc[:, 'month'].values[0] > 12:  # Überlauf des Monats
            X_next.loc[:, 'month'] = 1
            X_next.loc[:, 'year'] += 1
        
        # Aktualisiere Wochenzahl
        X_next.loc[:, 'week_of_year'] = (X_next['week_of_year'] % 52) + 1
        
        # Aktualisiere den Tag der Woche
        X_next.loc[:, 'DayOfWeek'] = (X_next['DayOfWeek'] % 7) + 1
        
        # Nutze die Promo-Informationen aus dem Testdatensatz für den jeweiligen Tag
        if day < len(test_data):  # Wenn der Testdatensatz Informationen für den Tag enthält
            X_next.loc[:, 'Promo'] = test_data.iloc[day]['Promo']
        else:
            X_next.loc[:, 'Promo'] = 0  # Standardwert, falls keine Informationen vorhanden sind
        
        # Optional: Aktualisiere andere Features (Open, StateHoliday etc.) basierend auf Testdaten
        if 'Open' in test_data.columns:
            X_next.loc[:, 'Open'] = test_data.iloc[day]['Open']
        if 'StateHoliday' in test_data.columns:
            X_next.loc[:, 'StateHoliday'] = test_data.iloc[day]['StateHoliday']

    return predictions

# TimeSeriesSplit für Cross-Validation
tscv = TimeSeriesSplit(n_splits=5)

# Überprüfung mit Cross-Validation
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

cv_rmspe_scores = []
for train_idx, test_idx in tscv.split(X_train):
    X_train_cv, X_test_cv = X_train.iloc[train_idx], X_train.iloc[test_idx]
    y_train_cv, y_test_cv = y_train.iloc[train_idx], y_train.iloc[test_idx]
    
    # Pipeline auf den Cross-Validation-Trainingsdaten fitten
    pipeline.fit(X_train_cv, y_train_cv)
    
    # Vorhersage für den Testdatensatz
    y_pred_cv = pipeline.predict(X_test_cv)
    
    # Berechne RMSPE für diesen Split
    rmspe_score = rmspe(y_test_cv, y_pred_cv)
    cv_rmspe_scores.append(rmspe_score)

# Ausgabe der Ergebnisse
mean_cv_rmspe = np.mean(cv_rmspe_scores)
print(f"Durchschnittliche RMSPE (Cross-Validation): {mean_cv_rmspe:.4f}")

# Nimm die letzten Daten als Startpunkt für die Vorhersage
X_last = X_test.iloc[-1]

# Vorhersage für die nächsten 7 Tage unter Verwendung von Promo-Informationen aus dem Testdatensatz
num_days = 7
test_data_next_days = X_test.iloc[:num_days]  # Extrahiere die Testdaten für die nächsten Tage

# Vorhersagen für die nächsten Tage machen
future_predictions = predict_multiple_days_with_updates(X_last, test_data_next_days, num_days)

# Ergebnisse anzeigen
print(f"Vorhersagen für die nächsten {num_days} Tage:", future_predictions)

Durchschnittliche RMSPE (Cross-Validation): 0.2424
Vorhersagen für die nächsten 7 Tage: [9299.0, 9005.0, 8796.0, 0.0, 0.0, 8942.0, 8649.0]


1 estimator:
Durchschnittliche RMSPE (Cross-Validation): 0.2424
Vorhersagen für die nächsten 7 Tage: [9299.0, 9005.0, 8796.0, 0.0, 0.0, 8942.0, 8649.0]

bestimmen genauigkeit vorhersage

In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import make_scorer
from sklearn.model_selection import TimeSeriesSplit

# RMSPE Funktion definieren
def rmspe(y_true, y_pred):
    y_pred = np.array(y_pred)  # Umwandlung der Vorhersagen in ein Numpy-Array
    mask = y_true != 0
    y_true_filtered = y_true[mask]
    y_pred_filtered = y_pred[mask]
    return np.sqrt(np.mean(((y_true_filtered - y_pred_filtered) / y_true_filtered) ** 2))

# RMSPE als Scorer definieren
rmspe_scorer = make_scorer(rmspe, greater_is_better=False)

# Laden des Datensatzes
data_cleaned = "../data/cleaned_train_alt.csv"
data = pd.read_csv(data_cleaned, delimiter=",", encoding="latin", header=0, thousands=",", decimal='.', low_memory=False)

# Zielvariable und Features definieren
X = data.drop('Sales', axis=1)
y = data['Sales']

# Definiere die numerischen und kategorischen Features
numerical_features = ['year', 'month', 'day', 'week_of_year', 'lag_1', 'lag_7']
already_encoded_features = ['Open', 'Promo', 'promo2']
categorical_features_to_encode = ['Store', 'DayOfWeek', 'StoreType', 'StateHoliday', 'Assortment']

# Preprocessing Pipeline für numerische und kategorische Features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features + already_encoded_features),
        ('enc', 'passthrough', already_encoded_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_to_encode)
    ])

# Modell für RandomForest erstellen
model = RandomForestRegressor(n_estimators=20, random_state=42)

# Pipeline erstellen
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

# Funktion zur Vorhersage mehrerer Tage mit Feature-Update
def predict_multiple_days_with_updates(X_last, test_data, num_days):
    predictions = []
    X_next = X_last.copy()

    # Wandle X_next in ein DataFrame um, falls es keine DataFrame ist
    X_next = pd.DataFrame([X_next])

    for day in range(num_days):
        # Vorhersage des nächsten Tages
        y_next_pred = pipeline.predict(X_next)[0]
        predictions.append(y_next_pred)
        
        # Aktualisiere die Lag-Features basierend auf der Vorhersage
        X_next.loc[:, 'lag_1'] = y_next_pred  # Verschiebe die vorhergesagte Sales als lag_1
        X_next.loc[:, 'lag_7'] = X_last['lag_1']  # Aktualisiere lag_7 mit den vorherigen lag_1 Werten
        
        # Aktualisiere andere Features
        X_next.loc[:, 'day'] += 1  # Verschiebe den Tag
        if X_next.loc[:, 'day'].values[0] > 31:  # Beispiel: Überlauf des Tages
            X_next.loc[:, 'day'] = 1
            X_next.loc[:, 'month'] += 1  # Verschiebe den Monat bei Tageswechsel
        if X_next.loc[:, 'month'].values[0] > 12:  # Überlauf des Monats
            X_next.loc[:, 'month'] = 1
            X_next.loc[:, 'year'] += 1
        
        # Aktualisiere Wochenzahl
        X_next.loc[:, 'week_of_year'] = (X_next['week_of_year'] % 52) + 1
        
        # Aktualisiere den Tag der Woche
        X_next.loc[:, 'DayOfWeek'] = (X_next['DayOfWeek'] % 7) + 1
        
        # Nutze die Promo-Informationen aus dem Testdatensatz für den jeweiligen Tag
        if day < len(test_data):  # Wenn der Testdatensatz Informationen für den Tag enthält
            X_next.loc[:, 'Promo'] = test_data.iloc[day]['Promo']
        else:
            X_next.loc[:, 'Promo'] = 0  # Standardwert, falls keine Informationen vorhanden sind
        
        # Optional: Aktualisiere andere Features (Open, StateHoliday etc.) basierend auf Testdaten
        if 'Open' in test_data.columns:
            X_next.loc[:, 'Open'] = test_data.iloc[day]['Open']
        if 'StateHoliday' in test_data.columns:
            X_next.loc[:, 'StateHoliday'] = test_data.iloc[day]['StateHoliday']

    return predictions

# TimeSeriesSplit für Cross-Validation
tscv = TimeSeriesSplit(n_splits=5)

# Überprüfung mit Cross-Validation
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

cv_rmspe_scores = []
for train_idx, test_idx in tscv.split(X_train):
    X_train_cv, X_test_cv = X_train.iloc[train_idx], X_train.iloc[test_idx]
    y_train_cv, y_test_cv = y_train.iloc[train_idx], y_train.iloc[test_idx]
    
    # Pipeline auf den Cross-Validation-Trainingsdaten fitten
    pipeline.fit(X_train_cv, y_train_cv)
    
    # Vorhersage für den Testdatensatz
    y_pred_cv = pipeline.predict(X_test_cv)
    
    # Berechne RMSPE für diesen Split
    rmspe_score = rmspe(y_test_cv.values, y_pred_cv)
    cv_rmspe_scores.append(rmspe_score)

# Ausgabe der Ergebnisse
mean_cv_rmspe = np.mean(cv_rmspe_scores)
print(f"Durchschnittliche RMSPE (Cross-Validation): {mean_cv_rmspe:.4f}")

# Nimm die letzten Daten als Startpunkt für die Vorhersage
X_last = X_test.iloc[-1]

# Vorhersage für die nächsten 7 Tage unter Verwendung von Promo-Informationen aus dem Testdatensatz
num_days = 7
test_data_next_days = X_test.iloc[:num_days]  # Extrahiere die Testdaten für die nächsten Tage

# Vorhersagen für die nächsten Tage machen
future_predictions = predict_multiple_days_with_updates(X_last, test_data_next_days, num_days)

# Ergebnisse anzeigen
print(f"Vorhersagen für die nächsten {num_days} Tage:", future_predictions)

# Tatsächliche Verkaufszahlen für die nächsten 7 Tage aus dem Testdatensatz extrahieren
actual_sales_next_days = y_test.iloc[:num_days].values

# Berechnung der RMSPE für die Vorhersagen
prediction_rmspe = rmspe(actual_sales_next_days, future_predictions)

# Ergebnis anzeigen
print(f"RMSPE der Vorhersagen für die nächsten {num_days} Tage: {prediction_rmspe:.4f}")


Durchschnittliche RMSPE (Cross-Validation): 0.2126
Vorhersagen für die nächsten 7 Tage: [9702.0, 9405.5, 8239.25, 0.0, 0.0, 10660.85, 10142.75]
RMSPE der Vorhersagen für die nächsten 7 Tage: 0.6569


Durchschnittliche RMSPE (Cross-Validation): 0.2126
Vorhersagen für die nächsten 7 Tage: [9702.0, 9405.5, 8239.25, 0.0, 0.0, 10660.85, 10142.75]
RMSPE der Vorhersagen für die nächsten 7 Tage: 0.6569

In [6]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import make_scorer
from sklearn.model_selection import TimeSeriesSplit

# RMSPE Funktion definieren
def rmspe(y_true, y_pred):
    y_pred = np.array(y_pred)  # Umwandlung der Vorhersagen in ein Numpy-Array
    mask = y_true != 0
    y_true_filtered = y_true[mask]
    y_pred_filtered = y_pred[mask]
    return np.sqrt(np.mean(((y_true_filtered - y_pred_filtered) / y_true_filtered) ** 2))

# RMSPE als Scorer definieren
rmspe_scorer = make_scorer(rmspe, greater_is_better=False)

# Laden des Datensatzes
data_cleaned = "../data/cleaned_train_alt.csv"
data = pd.read_csv(data_cleaned, delimiter=",", encoding="latin", header=0, thousands=",", decimal='.', low_memory=False)

# Zielvariable und Features definieren
X = data.drop('Sales', axis=1)
y = data['Sales']

# Definiere die numerischen und kategorischen Features
numerical_features = ['year', 'month', 'day', 'week_of_year', 'lag_1', 'lag_7']
already_encoded_features = ['Open', 'Promo', 'promo2']
categorical_features_to_encode = ['Store', 'DayOfWeek', 'StoreType', 'StateHoliday', 'Assortment']

# Preprocessing Pipeline für numerische und kategorische Features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('enc', 'passthrough', already_encoded_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_to_encode)
    ])

# Modell für RandomForest erstellen
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Pipeline erstellen
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

#Funktion zur vorhersage der nächsten x Tage
def predict_multiple_days_with_updates(X_last, test_data, num_days):
    predictions = []
    X_next = X_last.copy()
    X_next = pd.DataFrame([X_next])  # In DataFrame umwandeln
    last_lags = list(X_last[['lag_1']].values.flatten())  # Initialisiere letzte Lags

    for day in range(num_days):
        # Preprocess die Daten vor der Vorhersage
        X_next_preprocessed = pipeline.named_steps['preprocessor'].transform(X_next)
        
        # Vorhersage des nächsten Tages
        y_next_pred = pipeline.named_steps['model'].predict(X_next_preprocessed)[0]
        predictions.append(y_next_pred)

        # Aktualisiere die Lag-Features basierend auf der Vorhersage
        last_lags.append(y_next_pred)
        last_lags.pop(0)  # Aktualisiere Lag-Werte

        X_next.loc[:, 'lag_1'] = y_next_pred
        X_next.loc[:, 'lag_7'] = last_lags[0]

        # Aktualisiere andere Features (Tag, Monat, Woche)
        X_next.loc[:, 'day'] += 1
        if X_next.loc[:, 'day'].values[0] > 31:
            X_next.loc[:, 'day'] = 1
            X_next.loc[:, 'month'] += 1
        if X_next.loc[:, 'month'].values[0] > 12:
            X_next.loc[:, 'month'] = 1
            X_next.loc[:, 'year'] += 1

        X_next.loc[:, 'week_of_year'] = (X_next['week_of_year'] % 52) + 1
        X_next.loc[:, 'DayOfWeek'] = (X_next['DayOfWeek'] % 7) + 1

        # Promo- und andere Features aus Testdaten übernehmen
        if day < len(test_data):
            X_next.loc[:, 'Promo'] = test_data.iloc[day]['Promo']
            X_next.loc[:, 'Open'] = test_data.iloc[day]['Open']
            X_next.loc[:, 'StateHoliday'] = test_data.iloc[day]['StateHoliday']

    return predictions


# TimeSeriesSplit für Cross-Validation
tscv = TimeSeriesSplit(n_splits=5)

# Überprüfung mit Cross-Validation
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

cv_rmspe_scores = []
for train_idx, test_idx in tscv.split(X_train):
    X_train_cv, X_test_cv = X_train.iloc[train_idx], X_train.iloc[test_idx]
    y_train_cv, y_test_cv = y_train.iloc[train_idx], y_train.iloc[test_idx]

    # Pipeline auf den Cross-Validation-Trainingsdaten fitten
    pipeline.fit(X_train_cv, y_train_cv)

    # Vorhersage für den Testdatensatz
    y_pred_cv = pipeline.predict(X_test_cv)

    # Berechne RMSPE für diesen Split
    rmspe_score = rmspe(y_test_cv.values, y_pred_cv)
    cv_rmspe_scores.append(rmspe_score)

# Ausgabe der Ergebnisse
mean_cv_rmspe = np.mean(cv_rmspe_scores)
print(f"Durchschnittliche RMSPE (Cross-Validation): {mean_cv_rmspe:.4f}")

# Nimm die letzten Daten als Startpunkt für die Vorhersage
X_last = X_test.iloc[-60]

# Vorhersage für die nächsten 7 Tage unter Verwendung von Promo-Informationen aus dem Testdatensatz
num_days = 60
test_data_next_days = X_test.iloc[:num_days]  # Extrahiere die Testdaten für die nächsten Tage

# Vorhersagen für die nächsten Tage machen
future_predictions = predict_multiple_days_with_updates(X_last, test_data_next_days, num_days)

# Ergebnisse anzeigen
print(f"Vorhersagen für die nächsten {num_days} Tage:", future_predictions)

# Tatsächliche Verkaufszahlen für die nächsten 7 Tage aus dem Testdatensatz extrahieren
actual_sales_next_days = y_test.iloc[:num_days].values

# Berechnung der RMSPE für die Vorhersagen
prediction_rmspe = rmspe(actual_sales_next_days, future_predictions)

# Ergebnis anzeigen
print(f"RMSPE der Vorhersagen für die nächsten {num_days} Tage: {prediction_rmspe:.4f}")


Durchschnittliche RMSPE (Cross-Validation): 0.2386
Vorhersagen für die nächsten 60 Tage: [6868.0, 6235.0, 5694.0, 0.0, 0.0, 9615.0, 8180.0, 6480.0, 6328.0, 5470.0, 0.0, 4090.0, 3776.0, 3756.0, 3973.0, 3476.0, 3169.0, 0.0, 9615.0, 10744.0, 10041.0, 8735.0, 9165.0, 8373.0, 0.0, 15301.0, 25107.0, 16837.0, 18127.0, 16058.0, 16077.0, 0.0, 8324.0, 8328.0, 8328.0, 6355.0, 6594.0, 6588.0, 0.0, 4090.0, 3646.0, 3309.0, 2723.0, 3333.0, 2685.0, 0.0, 9615.0, 10311.0, 10216.0, 8735.0, 9165.0, 8373.0, 0.0, 4090.0, 3646.0, 3309.0, 2930.0, 3848.0, 4086.0, 0.0]
RMSPE der Vorhersagen für die nächsten 60 Tage: 0.9235


1 Baum, 7 Tage:
Durchschnittliche RMSPE (Cross-Validation): 0.2386
Vorhersagen für die nächsten 7 Tage: [6868.0, 6235.0, 5694.0, 0.0, 0.0, 9615.0, 8180.0]
RMSPE der Vorhersagen für die nächsten 7 Tage: 0.4642

1 Baum, 60 Tage:
Durchschnittliche RMSPE (Cross-Validation): 0.2386
Vorhersagen für die nächsten 60 Tage: [6868.0, 6235.0, 5694.0, 0.0, 0.0, 9615.0, 8180.0, 6480.0, 6328.0, 5470.0, 0.0, 4090.0, 3776.0, 3756.0, 3973.0, 3476.0, 3169.0, 0.0, 9615.0, 10744.0, 10041.0, 8735.0, 9165.0, 8373.0, 0.0, 15301.0, 25107.0, 16837.0, 18127.0, 16058.0, 16077.0, 0.0, 8324.0, 8328.0, 8328.0, 6355.0, 6594.0, 6588.0, 0.0, 4090.0, 3646.0, 3309.0, 2723.0, 3333.0, 2685.0, 0.0, 9615.0, 10311.0, 10216.0, 8735.0, 9165.0, 8373.0, 0.0, 4090.0, 3646.0, 3309.0, 2930.0, 3848.0, 4086.0, 0.0]
RMSPE der Vorhersagen für die nächsten 60 Tage: 0.9235

40 Bäume, 60 Tage: 
Durchschnittliche RMSPE (Cross-Validation): 0.2112
Vorhersagen für die nächsten 60 Tage: [7006.95, 6038.9, 5713.375, 0.0, 0.0, 10342.425, 11745.55, 9687.325, 8993.9, 7979.475, 0.0, 11497.0, 14109.975, 13406.45, 11244.675, 9955.425, 9238.6, 0.0, 9574.125, 10272.075, 9887.45, 8986.45, 8635.05, 8113.35, 0.0, 11631.925, 14490.2, 15139.7, 15196.65, 10676.875, 9921.975, 0.0, 7064.925, 6889.55, 11720.025, 9834.7, 9020.85, 7961.35, 0.0, 10148.875, 9110.1, 7964.1, 6368.0, 5449.375, 5042.275, 0.0, 8972.25, 9663.475, 10388.15, 9497.2, 8774.325, 7876.825, 0.0, 8724.725, 8219.7, 7496.45, 6704.05, 7345.7, 7603.85, 0.0]
RMSPE der Vorhersagen für die nächsten 60 Tage: 0.8358

Überprüfen wie Werte in X_test aussehen und wieviele Einträge vorhanden sind

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import make_scorer
from sklearn.model_selection import TimeSeriesSplit

# RMSPE Funktion definieren
def rmspe(y_true, y_pred):
    y_pred = np.array(y_pred)  # Umwandlung der Vorhersagen in ein Numpy-Array
    mask = y_true != 0
    y_true_filtered = y_true[mask]
    y_pred_filtered = y_pred[mask]
    return np.sqrt(np.mean(((y_true_filtered - y_pred_filtered) / y_true_filtered) ** 2))

# RMSPE als Scorer definieren
rmspe_scorer = make_scorer(rmspe, greater_is_better=False)

# Laden des Datensatzes
data_cleaned = "../data/cleaned_train_alt.csv"
data = pd.read_csv(data_cleaned, delimiter=",", encoding="latin", header=0, thousands=",", decimal='.', low_memory=False)

# Zielvariable und Features definieren
X = data.drop('Sales', axis=1)
y = data['Sales']

# Definiere die numerischen und kategorischen Features
numerical_features = ['year', 'month', 'day', 'week_of_year', 'lag_1', 'lag_7']
already_encoded_features = ['Open', 'Promo', 'promo2']
categorical_features_to_encode = ['Store', 'DayOfWeek', 'StoreType', 'StateHoliday', 'Assortment']


# Überprüfung mit Cross-Validation
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Nimm die letzten Daten als Startpunkt für die Vorhersage
X_last = X_test.iloc[0]

print("X_test")
print(len(X_test))
print("X_last")
print(X_last)


X_test
203442
X_last
Store                      1115
DayOfWeek                     2
Date                 2015-06-02
Open                          1
Promo                         1
StateHoliday                  0
year                       2015
month                         6
day                           2
week_of_year                 23
day_of_week                   1
is_weekend                    0
is_holiday                    0
is_school_holiday             0
StoreType                     d
Assortment                    c
promo2                        1
lag_1                    8409.0
lag_7                    6726.0
Name: 1017149, dtype: object
